In [0]:
# !pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
df = pd.read_csv('data/shoes_prices.csv', low_memory=False)

In [0]:
 def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
   X = df[feats].values
   y = df['prices_amountmin'].values

   scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
   return np.mean(scores), np.std(scores)

In [27]:
 df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
 run_model(['brand_cat'])

(-59.74991436028463, 5.857301054060102)

In [28]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-59.16236629345697, 5.83857552622226)

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    
    output_dict[key] = value
    
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [41]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()) )

len(keys)

476

In [46]:
def get_name_feat(key):
  return 'feat_'+key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [48]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_video game platform', 'feat_quantity in set',
       'feat_overall material', 'feat_date first available', 'feat_polarised',
       'feat_clothing size type', 'feat_type', 'feat_size/dimensions',
       'feat_lining material', 'feat_lens color family'],
      dtype='object', length=526)

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [56]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.46254497873733,
 'color': 47.62294188201941,
 'gender': 50.00545196816051,
 'manufacturer part number': 36.130192999672886,
 'material': 34.78900883218842}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key)+'_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [63]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[ df.brand != df.feat_brand][ ['brand','feat_brand'] ].shape 

(9496, 2)

In [0]:
feats = ['']

In [64]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-59.249826897962166, 5.831901993547003)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]
# feats_cat

In [0]:
feats = ['brand_cat','feat_brand_cat', 'feat_gender_cat','feat_fabric content_cat','feat_case thickness_cat','feat_age_cat','feat_sport_cat','feat_style_cat','feat_material_cat','feat_gemstone type_cat']
feats += [ ]
feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [94]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)
print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)

(-58.89179912776454, 6.264431399144455)


Weight,Feature
0.3214 ± 0.0064,brand_cat
0.0880 ± 0.0113,feat_material_cat
0.0380 ± 0.0050,feat_brand_cat
0.0286 ± 0.0037,feat_gender_cat
0.0178 ± 0.0023,feat_fabric content_cat
0.0142 ± 0.0068,feat_style_cat
0.0099 ± 0.0007,feat_age_cat
0.0048 ± 0.0004,feat_case thickness_cat
0.0022 ± 0.0003,feat_gemstone type_cat
0.0001 ± 0.0000,feat_sport_cat


In [78]:
df[ df['brand'] == 'nike' ].features_parsed.sample(5).values

array([{'style': 'athletic sneakers', 'condition': 'new with box'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'light lucid green/atomic mango', 'casual & dress shoe style': 'sandals', 'manufacturer part number': '631402303', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {},
       {'material': 'synthetic', 'gender': 'men', 'size': '9.5', 'shoe category': 'mens shoes', 'color': 'pink', 'model': '631047 801', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '631047 801', 'brand': 'nike', 'age group': 'adult', 'shoe width': 'medium (d, m)'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'brand': 'nike', 'color': 'light ash grey/black/photo blue', 'casual & dress shoe style': 'running shoes', 'shoe width': 'medium (d, m)'}],
      dtype=object)

In [80]:
df['feat_casual & dress shoe style'].value_counts()

fashion sneakers           269
athletic sneakers          233
loafers & slip ons         184
running shoes              180
oxfords                    173
                          ... 
flip-flop                    1
chukka boots                 1
water shoes                  1
knee high boots              1
running, cross training      1
Name: feat_casual & dress shoe style, Length: 111, dtype: int64